In [1]:
# import warnings
# warnings.filterwarnings('ignore',category=FutureWarning)
# import warnings
# warnings.filterwarnings("ignore")

import sys
#import os

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py as h5
import sklearn
from sklearn.multioutput import MultiOutputClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, precision_recall_curve, accuracy_score, confusion_matrix  
from sklearn.metrics import average_precision_score
import pickle
import keras
from keras.models import load_model
import tensorflow as tf
# tf.enable_eager_execution()
print(tf.__version__)
# import util

print("Python version:\n{}\n".format(sys.version))
print("matplotlib version: {}".format(matplotlib.__version__))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("sklearn version: {}".format(sklearn.__version__))
print("keras version: {}".format(keras.__version__))
import pickle as pkl

Using TensorFlow backend.


1.15.2
Python version:
3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]

matplotlib version: 3.5.1
pandas version: 1.3.5
numpy version: 1.21.5
sklearn version: 1.0.2
keras version: 2.2.4


In [2]:
model = load_model('deepbeat.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







2023-04-18 00:10:22.905289: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-04-18 00:10:22.913624: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2399980000 Hz
2023-04-18 00:10:22.914755: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55bdb52e4d20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-04-18 00:10:22.914782: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-04-18 00:10:22.919668: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-04-18 00:10:23.009709: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55bdb5317c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-18 00:10:23.009751: I tensorflow/compiler/xla/s




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [3]:
from scipy.signal import resample
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
from scipy.special import softmax

In [4]:
# testing_data_label_pairs = [
#     ['data_simband_ecg_2400.npy', 'label_simband.npy'],
#     ['data_simband_ppg_2400.npy', 'label_simband.npy'],
#     ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy'],
#     ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy'],
#     ['data_staford_2400.npy', 'label_staford.npy'],
#     ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy'],
#     ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy']
# ]

# test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
# for pair in testing_data_label_pairs:
    
#     print(pair[0].split('.')[0].split('/')[-1])
    
#     x_path = test_data_folder + pair[0]
#     y_path = test_data_folder + pair[1]
#     x_id = pair[0].split('.')[0][:-5]

#     x = np.load(x_path)
# #     y = np.load(y)
#     x_resampled = []

#     for x in tqdm(x):
#         resampled = resample(x, 800)
#         resampled = (resampled - np.min(resampled) / np.max(resampled) - np.min(resampled))
#         x_resampled.append(resampled)
        
#     x_resampled = np.asarray(x_resampled)
#     x_resampled = x_resampled.reshape(x_resampled.shape[0], x_resampled.shape[1], 1)

#     print(x_resampled.shape)
#     np.save(test_data_folder+x_id+'_800.npy', x_resampled)
        
        

#     print(x_path)
#     print(y_path)

In [5]:
testing_data_label_pairs = [
    ['data_simband_ecg_800.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_simband_ppg_800.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_ucla_ecg_800.npy', 'label_ucla_ecg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_ppg_800.npy', 'label_ucla_ppg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_staford_800.npy', 'label_staford.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_goodquality_800.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_badquality_800.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl']
]
test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]
    patient_info_path = pair[2]
    test_ds_name = pair[0].split('.')[0]
    print(x_path)
    print(y_path)

    x = np.load(x_path)
    y = np.load(y_path)
    
    PPG_out = model.predict(x)[1]
    PPG_preds = PPG_out.argmax(1)
    PPG_pred_probs = softmax(PPG_out, axis=1)[:, 1]
        
    all_targets = y
    
    precision, recall, thresholds = precision_recall_curve(all_targets, PPG_pred_probs)
    pr_auc = auc(recall, precision)

    print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets, PPG_preds), 4)}')
    print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets, PPG_pred_probs), 4)}')
    print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')
    
    
    rounding = 3
    if patient_info_path is not None:
        print(patient_info_path)
        patient_idx_dict = pkl.load(open(patient_info_path, 'rb'))
        unique_patients = list(patient_idx_dict.keys())
        all_bt_aurocs = []
        all_bt_auprcs = []
        all_bt_pred_probs = np.asarray([])
        all_bt_targets = np.asarray([])
        for i in tqdm(range(1000)):
            np.random.seed(i)
            random_sample_patients = np.random.choice(unique_patients, len(unique_patients), replace=True)
            sample_idx = []
            random_subsample_idx = []
            for patient in random_sample_patients:
                random_subsample_idx += patient_idx_dict[patient]
            random_subsample_idx = np.asarray(random_subsample_idx)

            auroc = roc_auc_score(all_targets[random_subsample_idx], PPG_pred_probs[random_subsample_idx])
            precision, recall, thresholds = precision_recall_curve(all_targets[random_subsample_idx], PPG_pred_probs[random_subsample_idx])
            auprc = auc(recall, precision)

            all_bt_pred_probs = np.concatenate((PPG_pred_probs[random_subsample_idx], all_bt_pred_probs))
            all_bt_targets = np.concatenate((all_targets[random_subsample_idx], all_bt_targets))
            all_bt_aurocs.append(auroc)
            all_bt_auprcs.append(auprc)

        all_bt_aurocs = np.asarray(all_bt_aurocs)
        all_bt_auprcs = np.asarray(all_bt_auprcs)
        
        print(all_bt_targets.shape, all_bt_pred_probs.shape)
        
        np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_deepbeat_aurocs.npy', all_bt_aurocs)
        np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_deepbeat_auprcs.npy', all_bt_auprcs)
        np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_deepbeat_targets.npy', all_bt_targets)
        np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_deepbeat_pred_probs.npy', all_bt_pred_probs)

        print(f'\t AUROC {round(np.mean(all_bt_aurocs), rounding)} [{round(np.mean(all_bt_aurocs) - 1.96 *  (np.std(all_bt_aurocs) / np.sqrt(len(all_bt_aurocs))) , rounding)} {round(np.mean(all_bt_aurocs) + 1.96 *  (np.std(all_bt_aurocs) / np.sqrt(len(all_bt_aurocs))) , rounding)}]')
        print(f'\t AUPRC {round(np.mean(all_bt_auprcs), rounding)} [{round(np.mean(all_bt_auprcs) - 1.96 *  (np.std(all_bt_auprcs) / np.sqrt(len(all_bt_auprcs))) , rounding)} {round(np.mean(all_bt_auprcs) + 1.96 *  (np.std(all_bt_auprcs) / np.sqrt(len(all_bt_auprcs))) , rounding)}]')

    print()
    
#     break

data_simband_ecg_800
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_800.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy


2023-04-18 00:10:44.605128: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-04-18 00:10:44.755302: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


[TEST] 	PPG      F1: 0.0792
[TEST] 	PPG ROC AUC: 0.6058
[TEST] 	PPG PR  AUC: 0.4959
/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl


  0%|          | 0/1000 [00:00<?, ?it/s]

(856988,) (856988,)
	 AUROC 0.605 [0.603 0.608]
	 AUPRC 0.498 [0.494 0.502]

data_simband_ppg_800
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_800.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.752
[TEST] 	PPG ROC AUC: 0.8706
[TEST] 	PPG PR  AUC: 0.8019
/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl


...

(250683755,) (250683755,)
	 AUROC 0.606 [0.603 0.609]
	 AUPRC 0.205 [0.201 0.208]

data_ucla_ppg_800
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_800.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.5728
[TEST] 	PPG ROC AUC: 0.8719
[TEST] 	PPG PR  AUC: 0.6156
/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl


  0%|          | 0/1000 [00:00<?, ?it/s]

(250683755,) (250683755,)
	 AUROC 0.872 [0.87 0.875]
	 AUPRC 0.607 [0.601 0.613]

data_staford_800
/labs/hulab/Robust_learning_TESTDATA/data_staford_800.npy
/labs/hulab/Robust_learning_TESTDATA/label_staford.npy
[TEST] 	PPG      F1: 0.6396
[TEST] 	PPG ROC AUC: 0.8835
[TEST] 	PPG PR  AUC: 0.7321
/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl


  0%|          | 0/1000 [00:00<?, ?it/s]

(17556623,) (17556623,)
	 AUROC 0.883 [0.882 0.884]
	 AUPRC 0.726 [0.723 0.73]



In [19]:
# mv /labs/hulab/Robust_learning_TESTDATA/data_ucl_800.npy /labs/hulab/Robust_learning_TESTDATA/trash/data_ucl_800.npy